# <center>Curso de Modelagem de Dados para IA - PARTE 8</center>

<img src="img/image.jpg" alt="Drawing" style="width: 300px;"/>


## Extração e Reorganização dos Dados
Às vezes, os dados que se deseja analisar chegam em um formato que não é o ideal para as necessidades ou contêm campos de dados adicionais que podem não ser interessantes. Nesse caso, convém pré-processar os dados para obtê-los em um formato adequado para análise posterior. Ilustramos alguns desses tipos de problemas aqui, no contexto dos dados do Twitter. Conforme observado anteriormente, os arquivos de dados do Twitter estão disponíveis por meio de links de download separados.

### Extraindo e reorganizando dados do Twitter
O formato padrão para os resultados retornados de uma pesquisa da API do Twitter é um objeto ***JSON (JavaScript Object Notation)*** contendo aproximadamente 40 campos diferentes. Para uma análise focada em retuítes, não são todos esses campos que são de interesse, mas apenas em um subconjunto, conforme descrito abaixo. A estratégia geral será usar os módulos *json* e *csv*, ambos parte da ***Python Standard Library***, para carregar os dados JSON iniciais dos arquivos, extrair campos de interesse e salvar o conjunto de dados reduzido em um arquivo CSV para processamento posterior. O módulo *json* contém o método <span style="font-family: 'Courier'">json.loads</span> que converte um objeto JSON em um dicionário Python. No código a seguir, lemos todos os tweets em um arquivo JSON, extraímos informações de interesse do dicionário resultante e gravamos esse subconjunto de dados de volta em um arquivo CSV, com uma linha por tweet. Um dos desafios de trabalhar com dados textuais como os encontrados em tweets é que eles podem conter caracteres de nova linha (<span style="font-family: 'Courier'">\n</span>) e retorno de carro (<span style="font-family: 'Courier'">\r</span>) que podem complicar o processamento de texto; no código abaixo, nós simplesmente substituímos cada um deles por um espaço para que não tenhamos que lidar com eles.

Para este exemplo, não será utilizado o conjunto de dados completo, pois é uma violação dos Termos de Serviço do Twitter compartilhar mais de 50.000 tweets em um dia. No entanto, embora o conjunto de dados completo contenha mais de 450.000 tweets, foir reduzido o tamanho total do arquivo de aproximadamente 3,5 Gb para 119 Mb, eliminando a maioria dos cerca de 40 campos que não são de interesse, aderindo assim ao espírito de intenção do Twitter ToS. Para este exemplo, são fornecidos dois arquivos no formato JSON contendo dados de tweet originais coletados com a API de streaming para se ter pelo menos uma noção de como o processo funciona.

Os campos escolhidos e que foram mantidos no CSV incluem: <span style="font-family: 'Courier'">id, created_at, lang, user screen_name, user created_at, user id, user followers_count, user friends_count, user time_zone, user utc_offset, retweeted_status, retweeted_status id, retweeted_status user screen_name, retweeted_status uer ID. O código abaixo lê todos os arquivos em uma pasta, extrai dados do subconjunto de campos e salva os resultados em um formato de valores separados por vírgula (CSV) em um arquivo chamado 'climatechange_tweets_sample.csv'.

In [5]:
import glob
import json
import csv
import pandas as pd

In [6]:
# Open CSV output files for reading and writing
input_dir = "data/twitter/"
output_dir = "data/twitter/"
hashtag = "climatechange"

# Open main twitter data CSV file and write header row
output_file = output_dir + hashtag + "_tweets_sample.csv"
f_out = open(output_file, 'w', encoding='utf-8')
rowwriter = csv.writer(f_out, delimiter=',', lineterminator='\n')
outputrow = ['tweet_id','tweet_created_at','language','user_screen_name','user_created_at','user_id','followers_count','friends_count','time_zone','utc_offset','retweeted_status','retweet_id','retweet_user_screen_name','retweet_user_id','text']
rowwriter.writerow(outputrow)

# Define variables
inc = 0

files = glob.glob(input_dir + '*.json')
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        print("Working on file:" + file)
        for line in f:
            tweet = json.loads(line)
            if 'user' in tweet:
                
                # Set standard variables equal to tweet data
                tweet_id = tweet['id']
                tweet_created_at = tweet['created_at']
                text = tweet['text'].replace('\r', ' ').replace('\n', ' ')
                language = tweet['lang']
                user_screen_name = tweet['user']['screen_name']
                user_created_at = tweet['user']['created_at']
                user_id = tweet['user']['id']
                followers_count = tweet['user']['followers_count']
                friends_count = tweet['user']['friends_count']
                utc_offset = tweet['user']['utc_offset']
                time_zone = tweet['user']['time_zone']

                # Check if a retweet else original tweet
                if 'retweeted_status' in tweet:
                    retweeted_status = 1
                    retweet_id = tweet['retweeted_status']['id']
                    retweet_user_screen_name = tweet['retweeted_status']['user']['screen_name']
                    retweet_user_id = tweet['retweeted_status']['user']['id']
                else:
                    retweeted_status = 0
                    retweet_id = "None"
                    retweet_user_screen_name = "None"
                    retweet_user_id = "None"

                # Write to main output file
                outputrow = [str(tweet_id), tweet_created_at, language, user_screen_name, user_created_at, str(user_id), str(followers_count), str(friends_count), time_zone, utc_offset, str(retweeted_status), str(retweet_id), retweet_user_screen_name, str(retweet_user_id), text] 
                rowwriter.writerow(outputrow)

                inc += 1
                # Optional counter increments variables to track progress, useful for very large files.
                if inc%10000 == 0:
                    print(inc)

# Close the output file
f_out.close()

Working on file:data/twitter/climatechange_2018_11_27_11_30_06_972631.json
10000
20000
Working on file:data/twitter/climatechange_2018_11_26_17_19_15_679824.json
30000
40000


### Ajustando o arquivo CSV para acomodar timestamps de data/hora
O arquivo CSV que acabou de ser escrito em disco é um extrato muito menor dos dados de nosso interesse, mas queremos ajustá-lo um pouco para que possamos trabalhar com ele em Pandas com mais facilidade.

Dois dos campos no arquivo CSV representam timestamps de data/hora: <span style="font-family: 'Courier'">tweet_created_at</span> e <span style="font-family: 'Courier'">user_created_at</span>. A função <span style="font-family: 'Courier'">pd.read_csv</span> pode receber um argumento adicional indicando que colunas específicas devem ser analisadas como datas (*datetimes*), em vez de strings simples. Infelizmente, os dados do Twitter que extraímos do arquivo JSON estão em um formato que não é o formato padrão de data e hora, por exemplo, Thu Nov 29 19:22:55 +0000 2018. Pandas tem uma função chamada <span style="font-family: 'Courier'">to_datetime</span> que pode não apenas converter strings para objetos datetime, mas pode inferir datetimes de vários formatos diferentes. Essa inferência pode ser bastante lenta para um arquivo grande, no entanto.

Felizmente, a função <span style="font-family: 'Courier'">pd.to_datetime</span> pode ser fornecida com uma string de formato específica, para que ela não precise inferir um formato. Ao fornecer uma string de formato explícito para a função de conversão para nossos timestamps de data e hora de tweet, a conversão é bastante rápida (alguns segundos, mesmo para o arquivo de dados completo). As strings de formato são baseadas nos códigos de formato <span style="font-family: 'Courier'">strftime</span> do Python. Se não fornecermos uma dica de formato desse tipo, a conversão levará mais de um minuto para cada chamada de função. No código a seguir, nós:

- Fazemos a leitura do arquivo CSV em Pandas
- Convertemos dois dos campos que são timestamps de data e hora em objetos de data e hora com uma string de formato específico
- Escrevemos um novo arquivo CSV com os dados reformatados

In [7]:
tweet_df = pd.read_csv(output_dir + 'climatechange_tweets_sample.csv')
tweet_df.tweet_created_at = pd.to_datetime(tweet_df.tweet_created_at, format='%a %b %d %H:%M:%S +0000 %Y')
tweet_df.user_created_at = pd.to_datetime(tweet_df.user_created_at, format='%a %b %d %H:%M:%S +0000 %Y')
tweet_df.to_csv(output_dir + 'climatechange_tweets_fixed.csv', index=False)

### Trabalhando mais com o arquivo CSV reformatado
Tendo ajustado os formatos de data e hora, se quisermos ler nosso novo arquivo CSV em um dataframe pandas, podemos aumentar a chamada <span style="font-family: 'Courier'">pd.read_csv</span> para especificar quais colunas analisar como datas, usando a opção <span style="font-family: 'Courier'">parse_dates</span>. Se não tivéssemos alterado previamente os formatos de data e hora, esta chamada para <span style="font-family: 'Courier'">pd.read_csv</span> seria muito lenta devido à necessidade de fazer inferência de formato de data e hora. Agora que corrigimos os formatos, a análise da data prossegue rapidamente. A função <span style="font-family: 'Courier'">pd.read_csv</span>, no entanto, não permite especificar uma string de formato, por isso precisamos fazer a conversão com a função <span style="font-family: 'Courier'">pd.to_datetime</span> como acima. Veja como é a nova chamada de função <span style="font-family: 'Courier'">read_csv</span>:

In [8]:
tweet_df = pd.read_csv(output_dir + 'climatechange_tweets_fixed.csv', \
                       parse_dates=['tweet_created_at', 'user_created_at'])

In [9]:
tweet_df

,tweet_id,tweet_created_at,language,user_screen_name,user_created_at,user_id,followers_count,friends_count,time_zone,utc_offset,retweeted_status,retweet_id,retweet_user_screen_name,retweet_user_id,text
0,1067485738931908608,2018-11-27 18:30:07,en,mrbellavia,2008-03-11 03:18:15,14119938,2508,2564,NaN,NaN,0,None,None,None,"Of course. Why would he say anything else. ""Tr..."
1,1067485742782472192,2018-11-27 18:30:08,en,LesleyRumary,2011-05-29 14:31:50,307372049,167,260,NaN,NaN,1,1067440246281900032,Coffeewarblers,311533910,RT @Coffeewarblers: Climate change: CO2 emissi...
2,1067485743466057728,2018-11-27 18:30:08,und,nicolea19082597,2018-09-11 16:55:35,1039558080739135494,5,24,NaN,NaN,0,None,None,None,😬😬 #NaturalPhenomena #ElNiño #ClimateChange #...
3,1067485753662414855,2018-11-27 18:30:11,en,NLassandrello,2015-07-31 00:44:05,3396454306,108,52,NaN,NaN,1,1067473981626859523,wildlifeaction,22819917,RT @wildlifeaction: The National Climate Asses...
4,1067485757659643904,2018-11-27 18:30:12,en,thom_lydia,2018-05-08 16:49:47,993895751003623425,165,168,NaN,NaN,0,None,None,None,"""UN report says that unless governments act no..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44905,1067485674205515777,2018-11-27 18:29:52,en,EvEvangelist,2018-02-19 17:43:56,965643123144654848,19,89,NaN,NaN,0,None,None,None,"Oh. Dear : 2 ears, 1 mouth . _ for a reason. K..."
44906,1067485678420852736,2018-11-27 18:29:53,en,MichaelFairfax1,2013-09-07 16:25:59,1755799957,41,401,NaN,NaN,1,1067399041519157248,HarryPotterMAGE,828038191974346752,RT @HarryPotterMAGE: There are 3 urgent intern...
44907,1067485679507202049,2018-11-27 18:29:53,en,porridgeisgood,2014-09-09 19:38:28,2800466323,3714,2535,NaN,NaN,1,1067475114436919296,JWSpry,23023227,RT @JWSpry: HUNDREDS More Frozen Turtles Since...
44908,1067485682925543425,2018-11-27 18:29:54,en,secularjen,2012-02-24 14:18:41,501817486,5645,4582,NaN,NaN,0,None,None,None,#ClimateChange: CO2 emissions rising for first...
